In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dateutil
from sklearn import ensemble,metrics,linear_model,tree
from tqdm import tqdm

# Read All data

In [2]:
campData=pd.read_csv('campaign_data.csv')
coupon_item_mappingData=pd.read_csv('coupon_item_mapping.csv')
customer_demographicsData=pd.read_csv('customer_demographics.csv')
c_trans_Data=pd.read_csv('customer_transaction_data.csv')
item_dataData=pd.read_csv('item_data.csv')
trainData=pd.read_csv('train.csv')
test_QyjYwdjData=pd.read_csv('test_QyjYwdj.csv')

In [3]:
trainData['dtType']='train'
test_QyjYwdjData['dtType']='test'

allData=trainData.append(test_QyjYwdjData,sort=False)
allData.shape

(128595, 6)

In [4]:
trainData['redemption_status'].sum()

729

In [5]:
campData['start_date']=campData['start_date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))
campData['end_date']=campData['end_date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))
campData['duration']=campData['end_date']-campData['start_date']
campData['duration']=campData['duration'].apply(lambda x: x.days)
campData['start_Month']=campData['start_date'].apply(lambda x: x.month)
campData['end_Month']=campData['end_date'].apply(lambda x: x.month)

In [6]:
c_trans_Data['date']=c_trans_Data['date'].apply(lambda x: dateutil.parser.parse(x,dayfirst=True))

In [7]:
print (max(c_trans_Data['date']),min(c_trans_Data['date']))
print (max(campData['start_date']),min(campData['start_date']))
print (max(campData['end_date']),min(campData['end_date']))

2013-12-06 00:00:00 2012-01-02 00:00:00
2013-10-21 00:00:00 2012-08-12 00:00:00
2013-12-20 00:00:00 2012-09-21 00:00:00


In [8]:
campData.sort_values('start_date')

,campaign_id,campaign_type,start_date,end_date,duration,start_Month,end_Month
27,26,X,2012-08-12,2012-09-21,40,8,9
26,27,Y,2012-08-25,2012-10-27,63,8,10
25,28,Y,2012-09-16,2012-11-16,61,9,11
24,29,Y,2012-10-08,2012-11-30,53,10,11
23,30,X,2012-11-19,2013-01-04,46,11,1
21,1,Y,2012-12-12,2013-01-18,37,12,1
22,2,Y,2012-12-17,2013-01-18,32,12,1
18,3,Y,2012-12-22,2013-02-16,56,12,2
20,4,Y,2013-01-07,2013-02-08,32,1,2
19,5,Y,2013-01-12,2013-02-15,34,1,2


In [9]:
for j,k,l in tqdm(zip(campData['start_date'],campData['end_date'],campData['campaign_id'])):
    c_trans_Data['campaign_'+str(l).zfill(2)]=c_trans_Data['date'].apply(lambda x:1 if ((x >= j) & (x <= k)) else 0)
#     if (c_trans_Data['date'][1324561] >= j) & (c_trans_Data['date'][1324561] <= k):
#         print (j,k)

28it [01:30,  3.20s/it]


In [10]:
campCol=['campaign_24', 'campaign_25',
       'campaign_20', 'campaign_23', 'campaign_21', 'campaign_22',
       'campaign_18', 'campaign_19', 'campaign_17', 'campaign_16',
       'campaign_13', 'campaign_11', 'campaign_12', 'campaign_10',
       'campaign_09', 'campaign_08', 'campaign_07', 'campaign_06',
       'campaign_03', 'campaign_05', 'campaign_04', 'campaign_01',
       'campaign_02', 'campaign_30', 'campaign_29', 'campaign_28',
       'campaign_27', 'campaign_26']

In [11]:
campTran=c_trans_Data[campCol]

campTran.iloc[1324544][campTran.iloc[1324544]==1].index

campTran['campParticipated']=campTran.apply(lambda x: x[x==1].index.values, axis = 1)

C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [12]:
c_trans_Data['campParticipated']=campTran['campParticipated']
c_trans_Data['filt']=c_trans_Data['campParticipated'].apply(lambda x: len(x))
for j in campCol:
    del c_trans_Data[j]

In [13]:
c_trans_Data.shape

(1324566, 9)

In [14]:
c_trans_Data=c_trans_Data[c_trans_Data['filt']!=0]
c_trans_Data['campaign_id']=c_trans_Data['campParticipated'].apply(lambda x: int(x[0].split('_')[1]))
del c_trans_Data['filt']
del c_trans_Data['campParticipated']

In [15]:
mergedData=pd.merge(c_trans_Data,campData,on='campaign_id',how='left')

In [16]:
mergedData['occurenceofPurch']=mergedData['date']-mergedData['start_date']
mergedData['occurenceofPurch']=mergedData['occurenceofPurch'].apply(lambda x: x.days)

In [17]:
mergedData.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,campaign_id,campaign_type,start_date,end_date,duration,start_Month,end_Month,occurenceofPurch
0,2012-09-01,1497,9281,4,71.24,0.00,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7
1,2012-09-01,1497,18919,2,284.25,-103.30,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7
2,2012-09-01,1497,19849,1,44.17,0.00,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7
3,2012-09-01,1497,23126,2,106.86,-34.91,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7
4,2012-09-01,1497,30874,1,88.69,0.00,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7


In [18]:
bohotData=pd.merge(mergedData,coupon_item_mappingData,on='item_id',how='left')
# bohotData=pd.merge(bohotData,campData,on='item_id',how='left')
bohotData.shape

(1761105, 16)

In [22]:
bohotData.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,campaign_id,campaign_type,start_date,end_date,duration,start_Month,end_Month,occurenceofPurch,coupon_id
0,2012-09-01,1497,9281,4,71.24,0.0,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7,NaN
1,2012-09-01,1497,18919,2,284.25,-103.3,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7,1043.0
2,2012-09-01,1497,19849,1,44.17,0.0,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7,24.0
3,2012-09-01,1497,19849,1,44.17,0.0,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7,8.0
4,2012-09-01,1497,19849,1,44.17,0.0,0.0,27,Y,2012-08-25,2012-10-27,63,8,10,7,33.0


In [23]:
computedFeature=bohotData.groupby(['campaign_id','coupon_id','customer_id'],as_index=False).agg({'quantity':{'totalQ':'sum','averQ':'mean'},
                                                                                                   'selling_price':{'totalSP':'sum','averSP':'mean'},
                                                                                                   'other_discount':{'totalOD':'sum','avgOD':'mean'},
                                                                                                   'coupon_discount':{'totalCD':'sum','avgCD':'mean'},
                                                                                                   'occurenceofPurch':{'avgOP':'mean'}

                                                                                                          })

C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [24]:
computedFeature.columns=[''.join(i) for i in computedFeature.columns]

In [25]:
computedFeature['campaign_id']=computedFeature['campaign_id'].map(int)
computedFeature['coupon_id']=computedFeature['coupon_id'].map(int)

In [26]:
trainData.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType
0,1,13,27,1053,0,train
1,2,13,116,48,0,train
2,6,9,635,205,0,train
3,7,13,644,1050,0,train
4,9,8,1017,1489,0,train


In [27]:
computedFeature.head()

,campaign_id,coupon_id,customer_id,quantitytotalQ,quantityaverQ,selling_pricetotalSP,selling_priceaverSP,other_discounttotalOD,other_discountavgOD,coupon_discounttotalCD,coupon_discountavgCD,occurenceofPurchavgOP
0,1,1,72,1,1.0,71.24,71.24,-35.26,-35.26,0.0,0.0,9.0
1,1,1,133,1,1.0,71.24,71.24,-35.26,-35.26,0.0,0.0,5.0
2,1,1,162,1,1.0,71.24,71.24,-35.26,-35.26,0.0,0.0,1.0
3,1,1,214,1,1.0,71.24,71.24,-35.26,-35.26,0.0,0.0,6.0
4,1,1,314,3,1.0,213.72,71.24,-105.78,-35.26,0.0,0.0,2.0


In [28]:
allNewFeatureData=pd.merge(allData,computedFeature,on=['campaign_id','coupon_id','customer_id'],how='left').fillna(0)

In [29]:
allNewFeatureData['redemption_status'].sum()

729.0

In [30]:
allNewFeatureData.shape

(128595, 15)

In [31]:
allNewFeatureData.head(2)

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType,quantitytotalQ,quantityaverQ,selling_pricetotalSP,selling_priceaverSP,other_discounttotalOD,other_discountavgOD,coupon_discounttotalCD,coupon_discountavgCD,occurenceofPurchavgOP
0,1,13,27,1053,0.0,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,13,116,48,0.0,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
allNewFeatureData['coupon_discountAvail']=allNewFeatureData['coupon_discountavgCD'].apply(lambda x: 1 if x !=0 else 0)

In [36]:
allNewFeatureData.head(2)

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType,quantitytotalQ,quantityaverQ,selling_pricetotalSP,selling_priceaverSP,other_discounttotalOD,other_discountavgOD,coupon_discounttotalCD,coupon_discountavgCD,occurenceofPurchavgOP,coupon_discountAvail
0,1,13,27,1053,0.0,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,13,116,48,0.0,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [37]:
allNewFeatureData['coupon_discountAvail'].sum()

492

# Merge traindata with Campdata

In [38]:
campData.head(2)

,campaign_id,campaign_type,start_date,end_date,duration,start_Month,end_Month
0,24,Y,2013-10-21,2013-12-20,60,10,12
1,25,Y,2013-10-21,2013-11-22,32,10,11


In [39]:
trainCampDatamerge=pd.merge(allNewFeatureData,campData[['campaign_id','campaign_type','duration','start_Month','end_Month']],on='campaign_id',how='left')
trainCampDatamerge.shape,trainData.shape

((128595, 20), (78369, 6))

In [40]:
trainCampDatamerge.head(2)

,id,campaign_id,coupon_id,customer_id,redemption_status,dtType,quantitytotalQ,quantityaverQ,selling_pricetotalSP,selling_priceaverSP,other_discounttotalOD,other_discountavgOD,coupon_discounttotalCD,coupon_discountavgCD,occurenceofPurchavgOP,coupon_discountAvail,campaign_type,duration,start_Month,end_Month
0,1,13,27,1053,0.0,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,X,47,5,7
1,2,13,116,48,0.0,train,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,X,47,5,7


## Feature engg in coupon mapping data and item data and transaction data

### merge mapping dtaa and transaction data

In [41]:
itenACoupData=pd.merge(coupon_item_mappingData,item_dataData,on='item_id',how='left').sort_values('coupon_id')

In [42]:
itenACoupData.head(2)

,coupon_id,item_id,brand,brand_type,category
31428,1,47797,1475,Established,Natural Products
31427,1,67376,4700,Established,Natural Products


### feature from transaction data Item level

### merge item information from transaction to coupon item information

In [43]:
customerProfileData=c_trans_Data.groupby('customer_id',as_index=False).agg({'quantity':'sum','date':'nunique','item_id':'count',
                                                                        'selling_price':{'maxSP':'max','minSP':'min',
                                                                                        'averageSP':'mean'},
                                                                        'other_discount':{'maxOD':'max','minOD':'min',
                                                                                        'averageOD':'mean'},
                                                                        'coupon_discount':{'maxCD':'max','minCD':'min',
                                                                                        'averageCD':'mean','totalCDAvailed':'count'}})

customerProfileData.columns = ['_'.join(col) for col in customerProfileData.columns.values]

C:\Users\swsh\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [ ]:
datatoCluster=mergedItemCoupon.groupby('coupon_id',as_index=False).mean()[['coupon_id', 
       'selling_pricemean', 'coupon_discountdiscountCount',
       'coupon_discounttotalDisc', 'other_discountOTdiscountCount',
       'other_discountOTtotalDisc', 'dicountPerGiven', 'dicountOthPerGiven',
       'dicountGivenPerc', 'dicountOthPerGivenPerc']]

# Final data for model prep

In [ ]:
mergedItemCoupon.head()

In [ ]:
# del datatoCluster['probaItem']

In [ ]:
trainCampDatamerge.head(2)

In [ ]:
campData.head(2)

In [ ]:
X1=pd.merge(trainCampDatamerge,coupon_categFind,on='coupon_id',how='left')

# X2=pd.merge(X1,datatoCluster,on='coupon_id',how='left')
# X3=pd.merge(X2,pcaDataTOCoup,on='coupon_id',how='left')
# X4=pd.merge(X1,campData[''],on='campaign_id',how='left')
X5=pd.merge(X1,customer_demographicsData,on='customer_id',how='left')
# X6=pd.merge(X5,customerProfileData,left_on='customer_id',right_on='customer_id_',how='left')
# X2.head(2)

In [ ]:
customer_demographicsData.head()

In [ ]:
X5['marital_status']=X5['marital_status'].fillna('Other')
X5['rented']=X5['rented'].fillna(0)
X5['no_of_children']=X5['no_of_children'].fillna('0')
X5['age_range']=X5['age_range'].fillna('Other')
X5['family_size']=X5['family_size'].fillna('9999')
X5['income_bracket']=X5['income_bracket'].fillna(9999)

In [ ]:
X5.columns

In [ ]:
from sklearn import preprocessing,model_selection

In [ ]:
llP=preprocessing.LabelEncoder()
# X5[jj]=llP.fit_transform(X5['campaign_type'])

In [ ]:
X5[['age_range', 'marital_status', 'rented', 'family_size','category', 'brand', 'brand_type',
       'no_of_children', 'income_bracket','campaign_type']].head(2)

In [ ]:
# X5['no_of_children']=X5['no_of_children'].map(str)

In [ ]:
pd.unique(X5['family_size'])

In [ ]:
categoColumns=['age_range', 'marital_status', 'rented', 'family_size','category', 'brand', 'brand_type',
       'no_of_children', 'income_bracket','campaign_type']
# catData=pd.get_dummies(X5[categoColumns],prefix_sep='_',)
for jj in categoColumns:
    print (jj)
    llP=preprocessing.LabelEncoder()
    X5[jj]=llP.fit_transform(X5[jj])

In [ ]:
# newtData=pd.concat([X5,catData],sort=False,axis=1)
newtData=X5

In [ ]:
newtData.columns

In [ ]:
XVar=list(newtData.columns)
XVar.remove('id')
XVar.remove('redemption_status')
XVar.remove('dtType')
# XVar.remove('start_date')
# XVar.remove('end_date')
YVar='redemption_status'

In [ ]:
# for j in ['campaign_id', 'coupon_id', 'customer_id']:
#     XVar.remove(j)

In [ ]:
trainPart=newtData[newtData['dtType']=='train']
testPart=newtData[newtData['dtType']=='test']
trainPart.shape,testPart.shape

In [ ]:
XVar

In [ ]:
from sklearn import model_selection,metrics
import xgboost as xgb

## Train test Split Train, Val, Test set

In [ ]:
trainX,testHX,trainY,testHY=model_selection.train_test_split(trainPart[XVar],trainPart[YVar],test_size=.2,stratify =trainPart[YVar])
trainX.shape,testHX.shape,trainY.shape,testHY.shape

In [ ]:
trainX1,valX1,trainY1,valY1=model_selection.train_test_split(trainX,trainY,test_size=.2,stratify =trainY)
trainX1.shape,valX1.shape,trainY1.shape,valY1.shape

In [ ]:
modelR=xgb.XGBClassifier().fit(trainX1,trainY1)

In [ ]:
XVar

In [ ]:
t1=modelR.predict(trainX)
t2=modelR.predict(valX1)
t3=modelR.predict(testHX)
trainScore=modelR.predict_proba(trainX)[:,1]
testScore=modelR.predict_proba(valX1)[:,1]
test2Score=modelR.predict_proba(testHX)[:,1]

print ('train {} vs {} validation {} vs {} test {} vs {}'.format(sum(trainY),sum(t1),sum(valY1),sum(t2), sum(testHY),sum(t3)))
metrics.roc_auc_score(trainY,trainScore),metrics.roc_auc_score(valY1,testScore),metrics.roc_auc_score(testHY,test2Score)

In [ ]:
import lightgbm  as lgb

In [ ]:
modelL=lgb.LGBMClassifier().fit(trainX1,trainY1)

In [ ]:
t1=modelL.predict(trainX)
t2=modelL.predict(valX1)
t3=modelL.predict(testHX)
trainScore=modelL.predict_proba(trainX)[:,1]
testScore=modelL.predict_proba(valX1)[:,1]
test2Score=modelL.predict_proba(testHX)[:,1]

print ('train {} vs {} validation {} vs {} test {} vs {}'.format(sum(trainY),sum(t1),sum(valY1),sum(t2), sum(testHY),sum(t3)))
metrics.roc_auc_score(trainY,trainScore),metrics.roc_auc_score(valY1,testScore),metrics.roc_auc_score(testHY,test2Score)

In [ ]:
params={'base_score':0.9,
        'booster':'gbtree', 
        'colsample_bylevel':1,
       'colsample_bytree':1, 
        'gamma':0, 
        'learning_rate':0.1, 
        'max_delta_step':0,
       'max_depth':3, 
        'min_child_weight':1, 
        'n_estimators':100,
        'eval_metric': ['logloss','auc'],
       'n_jobs':1,  'objective':'binary:logistic', 'random_state':0,
       'reg_alpha':0, 'reg_lambda':1, 'scale_pos_weight':1, 'subsample':1,
       'early_stopping_rounds' : 50,
       'stratified':True }

In [ ]:
num_round = 1000

dtrain = xgb.DMatrix(trainX1,trainY1)
# dtrain.set_group(Xg)

dvalid = xgb.DMatrix(valX1, valY1)
# dvalid.set_group(Xgt)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
bst = xgb.train(params, dtrain, num_round, evals = watchlist,
    early_stopping_rounds=50,verbose_eval=50,maximize=True)


In [ ]:
dtest = xgb.DMatrix(testHX)

In [ ]:
testScoreP=bst.predict(dtest)

In [ ]:
metrics.roc_auc_score(testHY,testScoreP)

In [ ]:
dtestOrg = xgb.DMatrix(testPart[XVar])
testScoreP2=bst.predict(dtestOrg)

In [ ]:
testScoreP2

In [ ]:
# testFScore=modelR.predict_proba(testPart[XVar])[:,1]
firstSub2=pd.DataFrame(data={'id':test_QyjYwdjData['id'],YVar:testScoreP2})
firstSub2.to_csv('firstSub9.csv',index=False)

## LGB model

In [ ]:
modelL.fit(trainPart[XVar],trainPart[YVar])

In [ ]:
lgbScore=modelL.predict_proba(testPart[XVar])[:,1]
lgbScore2=modelL.predict(testPart[XVar])

In [ ]:
# testFScore=modelR.predict_proba(testPart[XVar])[:,1]
firstSub2=pd.DataFrame(data={'id':test_QyjYwdjData['id'],YVar:lgbScore})
firstSub2.to_csv('lgbSub3.csv',index=False)

## XGB model

In [ ]:
modelR=xgb.XGBClassifier().fit(trainPart[XVar],trainPart[YVar])
xgbScore=modelL.predict_proba(testPart[XVar])[:,1]
xgbScore2=modelL.predict(testPart[XVar])

In [ ]:
# testFScore=modelR.predict_proba(testPart[XVar])[:,1]
firstSub2=pd.DataFrame(data={'id':test_QyjYwdjData['id'],YVar:xgbScore})
firstSub2.to_csv('xgbSub3.csv',index=False)